### Downloading a high-quality TTS dataset for German language (LibriTTS-style dataset)

In [ ]:
# HUI-Audio-Corpus-German: A high quality TTS dataset (https://arxiv.org/pdf/2106.06309.pdf)
!wget https://opendata.iisys.de/opendata/Datasets/HUI-Audio-Corpus-German/dataset_clean/others_Clean.zip

### Unzip dataset

In [ ]:
!unzip ./others_Clean.zip -d ./dataset

### Adjust dataset structure so that all train and val wav files are located in an individual folder

In [ ]:
from glob import glob
import csv

transcribed_audio_samples = []

for folder in glob("others_dataset/*"):
  for subfolder in glob(f"{folder}/*"):
    with open(f"{subfolder}/metadata.csv") as csv_file:
        audio_samples = csv.reader(csv_file, delimiter='\n')
        for audio_sample in audio_samples:
            transcribed_audio_sample = ''.join(audio_sample)
            filename, transcription = transcribed_audio_sample.split('|')
            new_transcribed_audio_sample = f"wavs/{filename}.wav|{transcription}"
            transcribed_audio_samples.append((f"{subfolder}/wavs/{filename}.wav", new_transcribed_audio_sample))

### Divide dataset into train and val subsets

In [ ]:
import random

random.shuffle(all_transcribed_audio_samples)
num_train_samples = int(len(all_transcribed_audio_samples) * 0.85)

train_dataset = transcribed_audio_samples[:num_train_samples]
val_dataset = transcribed_audio_samples[num_train_samples:]

### Copy files to train and val folders

In [ ]:
!mkdir dataset
!mkdir dataset/wavs

In [ ]:
import shutil

train_val_list = [('train', train_dataset), ('val', val_dataset)]

for stage, dataset in train_val_list:
    sample_list = []
    for old_sample, new_sample in dataset:
        # copy audio sample to new location
        shutil.copyfile(old_sample, "./dataset/" + new_sample.split("|")[0])
        sample_list.append(new_sample)

    with open(f"./dataset/{stage}.txt", "w") as sample_file:
        sample_file.write("\n".join(sample_list))

### Zip altered dataset

In [ ]:
!zip -r hui_audio_corpus.zip ./dataset

### Alter fine-tuning code to allow special letters (e.g. ä,ö,ü,ß for German language)
---
##### Source: https://github.com/152334H/DL-Art-School
##### File: codes/models/audio/tts/tacotron2/text/cleaners.py
- Alter method english_cleaners to clean input text for your language (line 83)

##### File: codes/models/audio/tts/tacotron2/text/symbols.py
- Add special characters to \_letters (line 12)

##### Create File: experiments/custom_language_gpt.yml
- Copy file experiments/EXAMPLE_gpt.yml and rename it to custom_language_gpt.yml
- Change experiment name to custom_language_gpt (line 1)
- Change name to train_dataset (line 14)
- Change path to ../../dataset/train.txt (line 18)
- Add attribute *tokenizer_vocab: ../custom_language_tokenizer.json* (line 29)
- Change name to val_dataset (line 31)
- Change path to ../../dataset/val.txt (line 35)
- Add attribute *tokenizer_vocab: ../custom_language_tokenizer.json* (line 46)
- Change value to 5000 (line 128)

##### File: .gitignore
- Add statement *!experiments/custom_language_gpt.yml* (line 169)

##### File: codes/requirements.laxed.txt
- Change transformers to transformers==4.29.2 (line 39)

##### File: codes/utils/util.py
- Change to *from torch import inf* (line 25)

### Install all required modules and clone repo with altered fine-tuning code

In [ ]:
!git clone https://github.com/thisserand/DL-Art-School
%cd DL-Art-School/codes/
!pip install -r requirements.laxed.txt

### Download model weights for the VQ-VAE and Autoregressive Model (GPT-2)

In [ ]:
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O ../experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O ../experiments/autoregressive.pth

### Create text file containing all transcriptions as source to train a tokenizer

In [ ]:
transcriptions = ""
dataset_path = "../../dataset"

for stage in ["train", "val"]:
    with open(f'{dataset_path}/{stage}.txt') as f:
        for line in f.readlines():
            transcriptions += ' ' + line.split("|")[1].strip()

with open("transcriptions.txt", "w") as f:
  f.write(transcriptions.strip())

### Train Tokenizer

In [ ]:
import re
import torch
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from german_transliterate.core import GermanTransliterate


def text_cleaners(text):
  ###########################################
  # ToDo Adjust this code for your language #
  ###########################################
  text = GermanTransliterate().transliterate(text)
  text = text.lower()
  text = re.sub(re.compile(r'\s+'), ' ', text)
  text = text.replace('"', '')
  return text


def remove_extraneous_punctuation(word):
    replacement_punctuation = {
        '{': '(', '}': ')',
        '[': '(', ']': ')',
        '`': '\'', '—': '-',
        '—': '-', '`': '\'',
        'ʼ': '\''
    }
    replace = re.compile("|".join([re.escape(k) for k in sorted(replacement_punctuation, key=len, reverse=True)]), flags=re.DOTALL)
    word = replace.sub(lambda x: replacement_punctuation[x.group(0)], word)
    extraneous = re.compile(r'^[@#%_=\$\^&\*\+\\]$')
    word = extraneous.sub('', word)
    return word

with open('transcriptions.txt', 'r', encoding='utf-8') as at:
    ttsd = at.readlines()
    allowed_characters_re = re.compile(r'^[a-zäöüß!:;"/, \-\(\)\.\'\?ʼ]+$')

    def preprocess_word(word, report=False):
        word = text_cleaners(word)
        word = remove_extraneous_punctuation(word)
        if not bool(allowed_characters_re.match(word)):
            if report and word:
                print(f"REPORTING: '{word}'")
            return ''
        return word

    def batch_iterator(batch_size=1000):
        print("Processing ASR texts.")
        for i in range(0, len(ttsd), batch_size):
            yield [preprocess_word(t, True) for t in ttsd[i:i+batch_size]]

    trainer = BpeTrainer(special_tokens=['[STOP]', '[UNK]', '[SPACE]'], vocab_size=255)
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.train_from_iterator(batch_iterator(), trainer, length=len(ttsd))
    tokenizer.save('../custom_language_tokenizer.json')

### Make sure the waveform audio has a sampling rate of 22.05kHz
##### Install librosa using the following command (automatically installs soundfile as well):
##### *pip install librosa*

In [ ]:
import librosa
import soundfile as sf

def resample_wav_file(input_file, target_sampling_rate=22050):
    # Load audio file
    audio, sampling_rate = librosa.load(input_file, sr=None)

    # Check if the sampling rate matches the target
    if sampling_rate != target_sampling_rate:

        # Resample audio to the target sampling rate
        audio_resampled = librosa.resample(audio, orig_sr=sampling_rate, target_sr=target_sampling_rate)

        # Overwrite the input file with the resampled audio
        sf.write(input_file, audio_resampled, target_sampling_rate)

# Resample all audio samples to 22.05kHz
dataset_path = "../../dataset/wavs/"
for wav_file in glob(dataset_path + "*.wav"):
    resample_wav_file(input_file)

### Fine-Tune the Autoregressive Model

In [ ]:
!python3 train.py -opt ../experiments/custom_language_gpt.yml

### The fine-tuned autoregressive model can be found here:
##### **/DL-Art-School/experiments/custom_language_gpt/models/**

### Upload fine-tuned model to HuggingFace

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

hf_user_name = ""
repository_name = ""
repo_id = f"{hf_user_name}/{repository_name}"
fine_tuned_model_path = "../experiments/custom_language_gpt/models/2500_gpt.pth"
hf_auth_token = ""


api.create_repo(repo_id=repo_id, token=hf_auth_token, repo_type='model')

model_url = api.upload_file(
    path_or_fileobj="fine_tuned_model_path",
    path_in_repo="custom_language_gpt.pth",
    repo_id=repo_id,
    repo_type="model",
)

print(f"The fine-tuned model was uploaded to: {model_url}")

### Clone adjusted inference code for German language

In [ ]:
!git clone https://gitlab.com/tisserand13/tortoise-tts
%cd tortoise-tts
!pip install -e .

### Define example text and voice samples to assess generation quality

In [ ]:
example_texts = ["Das Trainieren von Sprachmodellen für neue Sprachen funktioniert äußerst gut!",
                 "Äpfel und Birnen sind gesund für den Körper.",
                 "Übermorgen fahren wir in die Berge.",
                 "Das Café um die Ecke serviert köstlichen Kuchen und Kaffee.",
                 "Ein leises Lachen verzaubert die Luft und lässt Herzen höher schlagen.",
                 "Inmitten des Blütenmeers wandern wir durch den zauberhaften Frühlingswald."
                ]

voices = ["tom", "train_atkins", "angie", "daniel", "deniro", "emma"]

### Generate speech

In [ ]:
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice
import torchaudio
import IPython

tts = TextToSpeech(kv_cache=True)

for voice, text in zip(voices, example_texts):
    voice_samples, _ = load_voice(voice)
    audio = tts.tts_with_preset(text, voice_samples=voice_samples, preset='fast')
    torchaudio.save(f'{voice}.wav', audio.squeeze(0), 24000)
    IPython.display.display(IPython.display.Audio(f'{voice}.wav'))

In [ ]:
import librosa

hop_length = 512
db_threshold = -45

def trim_audio(audio):
    generated_audio = audio.squeeze().numpy()
    audio_rms = librosa.feature.rms(y=generated_audio, frame_length=2048, hop_length=hop_length)[0]
    audio_db = librosa.power_to_db(rmse**2, ref=np.max)

    start_index, end_index = None, None
    for index, frame_rms in enumerate(y_values):
        if frame_rms >= db_threshold:
            if start_index is None:
                    start_index = index
        elif start_index is not None:
            end_index = index - 1
            break

    return audio.squeeze(0)[:, start_index*hop_length:end_index*hop_length]